## Practice - Twitter classifier
With the Tweet Corpus from two Twitter accounts (archives from Ariana Grande and Trump)

2) 
3) Set up and fit a linear model and predict which account an input tweet is from and its probability

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import json

### Helper functions
You may find these useful in the lab. Feel free to modify for your needs.

In [2]:
def print_examples(data, probs, label1, label2, n=10):
    percent = lambda x: "{}%".format(round(x*100, 1))
    
    for text, pred in list(zip(data, probs))[:n]:
        print("{}\n{}: {} / {}: {}\n{}".format(
            text,
            label1,
            percent(pred[0]),
            label2,
            percent(pred[1]),
            "-"*50  # to print a line
        ))
        
def predict(model, vectorizer, data, all_predictions=False):
    data = vectorizer.transform(data)
    if all_predictions:
        return model.predict_proba(data)
    else:
        return model.predict(data)

### Cleaning function
Create a simple text cleaning function, as tweets are sensitive to major reformatting. You may experiment with this statement!

In [31]:
def twitter_text_clean(text):
    tokens =  TweetTokenizer().tokenize(str(text).lower())
    stop = stopwords.words("english")
    tokens = [w for w in tokens if w.isalpha() and w.lower() not in stop and w.lower() != "rt"]
    return " ".join(tokens).strip()

### Fetch data from /twitter_data

In [32]:
def tweets(name, test_size=0.1):
    with open("twitter_data/{}.json".format(name), encoding="utf-8") as f:
        tweets = [t.get("text") for t in json.load(f)]
        cleaned = [twitter_text_clean(t) for t in tweets]
        
        return train_test_split(cleaned, test_size=test_size)

In [33]:
# TODO: experiment with this parameter!
"""
initially gather a train and test set of each tweet file.
below, we use train to create another test set to evaluate the model

this means the two test datasets below \
are completely unseen to the model we train
"""
test_split = 0.2
ariana_train, ariana_test = tweets("ariana", test_size=test_split)
trump_train, trump_test = tweets("trump", test_size=test_split)

y = [1]*len(ariana_train) + [0]*len(trump_train)
x = ariana_train + trump_train
print("Train samples: {}".format(len(x)))

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=4310)

Train samples: 320


### TF-IDF + logistic regression
- Vectorize the tweets (e.g. with Count Vectorizer or TF-IDF Vectorizer).
- Logistic regression is neat because it spits out whether something is true or not.
    - This is exactly what we want in this case, to determine between two types of tweet sources (1 or 0).

In [34]:
vectorizer = TfidfVectorizer()
# define regression and fit
LR = LogisticRegression()
LR.fit(vectorizer.fit_transform(X_train), y_train)

# evaluate by confusion matrix
y_pred = predict(LR, vectorizer, X_test)
print(confusion_matrix(y_test, y_pred))

[[11  5]
 [ 2 14]]


In [35]:
ariana_prob = predict(LR, vectorizer, ariana_test, all_predictions=True)
print_examples(ariana_test, ariana_prob, "Trump", "Ariana")

heaven sent u
Trump: 50.3% / Ariana: 49.7%
--------------------------------------------------
ari album check table get chance also listen whole album proud
Trump: 34.5% / Ariana: 65.5%
--------------------------------------------------
arianators get position new
Trump: 45.8% / Ariana: 54.2%
--------------------------------------------------
alt physical cover positions
Trump: 25.0% / Ariana: 75.0%
--------------------------------------------------
miss u sending love got
Trump: 25.4% / Ariana: 74.6%
--------------------------------------------------
ya motive
Trump: 47.7% / Ariana: 52.3%
--------------------------------------------------
three days
Trump: 41.1% / Ariana: 58.9%
--------------------------------------------------
florida voters deadline register vote extended tonight register vote
Trump: 32.4% / Ariana: 67.6%
--------------------------------------------------
hello thank u favorite album yet response music made heart incredibly ful
Trump: 28.7% / Ariana: 71.3%
---------

In [36]:
trump_prob = predict(LR, vectorizer, trump_test, all_predictions=True)
print_examples(trump_test, trump_prob, "Trump", "Ariana")

president departs new jersey
Trump: 69.7% / Ariana: 30.3%
--------------------------------------------------
take care people fault plague came china
Trump: 56.2% / Ariana: 43.8%
--------------------------------------------------
obamacare replaced much better far cheaper alternative terminated supreme court
Trump: 60.2% / Ariana: 39.8%
--------------------------------------------------
authorized tens millions vaccine doses available month hundreds millions qui
Trump: 61.6% / Ariana: 38.4%
--------------------------------------------------
another million rapid tests support efforts reopen economies quickly possible including te
Trump: 61.2% / Ariana: 38.8%
--------------------------------------------------
pleased announce christopher c miller highly respected director national counterterrorism ce
Trump: 53.4% / Ariana: 46.6%
--------------------------------------------------
getting ready land great state florida see soon
Trump: 59.7% / Ariana: 40.3%
--------------------------------